In [ ]:
import torch
import pandas as pd
import os
from torchvision import models, transforms
from sklearn.neighbors import NearestNeighbors
from PIL import Image
import numpy as np
from csv import writer
import random

In [ ]:
#random 
with open('ratings.csv','a',newline='') as fd:
    x=[]
    for i in range(1000):
        x.append(random.random())
    for i in x:
        writer(fd).writerow([i])
        

In [ ]:
extracted_features = torch.load("extracted_features.pt")
user_preferance  = pd.read_csv('ratings.csv')
print(extracted_features.shape)
extracted_features=extracted_features[0:len(user_preferance)]
print(user_preferance.shape)
print(extracted_features.shape)
user_preferance = np.array(user_preferance)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
resnet50 = models.resnet50(weights=None)

resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])

resnet50.eval()

In [ ]:
def get_features(image_path):
    img = Image.open(image_path).convert('RGB')
    img = transform(img).unsqueeze(0) 
    with torch.no_grad():
        features = resnet50(img)
    return features.flatten().numpy()

In [ ]:
image_folder = "Pictures"
new_features = []
new_images = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]
for i in new_images:
    features = get_features(i)
    new_features.append(features)

In [ ]:
print(len(new_features))
new_features_np = np.array(new_features)
print(new_features_np.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
extracted_features = scaler.fit_transform(extracted_features)
new_features_np = scaler.transform(new_features_np)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
reduced_features = pca.fit_transform(extracted_features)
plt.scatter(reduced_features[:, 0], reduced_features[:, 1])
plt.show()

In [ ]:

model  = NearestNeighbors(n_neighbors=10)
model.fit(extracted_features)


In [ ]:
distances , indexes = model.kneighbors(new_features)
recommended = [] 
print(indexes[:5])
for i, path in enumerate(new_images):
    
    r = user_preferance[indexes[i]]
    a = np.mean(r)
    recommended.append((path,a))

In [ ]:
for img, avg_pref in recommended:
    print(f"Image: {img}, Recommended Preference: {avg_pref:.2f}")